In [33]:
import carla
import sys
import os
import time

old_chdir = os.getcwd()
os.chdir('/opt/carla-simulator/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.basic_agent import BasicAgent
from agents.navigation.behavior_agent import BehaviorAgent
os.chdir(old_chdir)
# from agents.navigation.global_route_planner_dao import GlobalRoutePlannerDAO

In [ ]:
client = carla.Client("localhost", 2000)
client.set_timeout(20)
world = client.load_world('Town01')
amap = world.get_map()


In [9]:
sampling_resolution = 2
# dao = GlobalRoutePlannerDAO(amap, sampling_resolution)
grp = GlobalRoutePlanner(amap, sampling_resolution)
# grp.setup()
spawn_points = world.get_map().get_spawn_points()
a = carla.Location(spawn_points[50].location)
b = carla.Location(spawn_points[100].location)
w1 = grp.trace_route(a, b) # there are other funcations can be used to generate a route in GlobalRoutePlanner.
i = 0
for w in w1:
    if i % 10 == 0:
        world.debug.draw_string(w[0].transform.location, 'O', draw_shadow=False,
        color=carla.Color(r=255, g=0, b=0), life_time=120.0,
        persistent_lines=True)
    else:
        world.debug.draw_string(w[0].transform.location, 'O', draw_shadow=False,
        color = carla.Color(r=0, g=0, b=255), life_time=1000.0,
        persistent_lines=True)
    i += 1

In [60]:
def draw_route(routes, life_time=120):
    for route in routes:
        for w in route:
            world.debug.draw_string(w[0].transform.location, 'O', draw_shadow=False,
            color=carla.Color(r=255, g=0, b=0), life_time=life_time,
            persistent_lines=True)

In [59]:
def draw_all_spawnpoints(spawn_points):
    for i, sp in enumerate(spawn_points):
        world.debug.draw_string(
            sp.location,
            f'O{i}',  # Optional: add index label
            draw_shadow=False,
            color=carla.Color(r=0, g=255, b=0),  # Green
            life_time=500.0,
            persistent_lines=True
     )

Initializing vehicle

In [52]:
def destroy_all_vehicles():
    """
    Destroy all vehicles in the world.
    """
    actors = world.get_actors().filter('vehicle.*')
    for actor in actors:
        actor.destroy()
    print("All vehicles destroyed.")

In [ ]:
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*model3*')[0]  # You can choose another model

In [ ]:
def generate_route(spawn_points_ids, draw_debug_route=True):
    route = []
    for i in range(len(spawn_points_ids) - 1):
        route.append(grp.trace_route(spawn_points[spawn_points_ids[i]].location, spawn_points[spawn_points_ids[i + 1]].location))
    if draw_debug_route:
        draw_route(route, life_time=300)
    return route

: 

In [54]:
spectator = world.get_spectator()
spectator.set_transform(carla.Transform(spawn_points[route_ids[0]].location + carla.Location(z=50), carla.Rotation(pitch=-90)))

In [ ]:
destroy_all_vehicles()
def follow_route(vehicle_bp, spawn_points, route_ids, agent_behavior='normal', draw_debug_route=True):
    """
    Follow the route using the vehicle.
    """
    route = generate_route(route_ids, draw_debug_route=draw_debug_route)

    vehicle = world.spawn_actor(vehicle_bp, spawn_points[route_ids[0]])
    agent = BehaviorAgent(vehicle, behavior=agent_behavior)  # options: 'normal', 'aggressive', etc.
    agent.set_global_plan(route[0])  # Set the global plan for the agent
    route_count = 0

    try:
        while True:

            if agent.done():
                print(f"Path up to {route_ids[route_count + 1]} completed.")
                if route_count == len(route) - 1:
                    print("Route completed.")
                    break
                
                route_count += 1
                agent.set_global_plan(route[route_count])  # Update the global plan for the agent
                print(f"New route set from {route_ids[route_count]} to {route_ids[route_count + 1]}.")

            control = agent.run_step()
            vehicle.apply_control(control)
            
            world.tick()

    finally:
        vehicle.destroy()


All vehicles destroyed.
Path up to 239 completed.
New route set from 239 to 138.
Path up to 138 completed.
New route set from 138 to 20.
Path up to 20 completed.
New route set from 20 to 38.
Path up to 38 completed.
Route completed.


In [30]:
def get_closest_spawnpoint(target_transform):
    """
    Find the closest spawn point to the given transform.
    :param transform: The transform to find the closest spawn point to.
    :return: The closest spawn point.
    """
    spawn_points = world.get_map().get_spawn_points()

    closest_spawn_point = min(spawn_points, key=lambda sp: sp.location.distance(target_transform.location))

    return closest_spawn_point

In [25]:
# Get all environment actors with semantic tags
environment_objects = world.get_environment_objects(carla.CityObjectLabel.Buildings)

print(f"Found {len(environment_objects)} buildings.")

for i, obj in enumerate(environment_objects):
    print(f"Building {i}:")
    print(f"  ID: {obj.id}")
    print(f"  Bounding box location: {obj.bounding_box.location}")
    print(f"  Bounding box extent: {obj.bounding_box.extent}")
    print(f"  Tags: {obj.type}")  # This is usually 'building' but may vary

Found 481 buildings.
Building 0:
  ID: 13914322317970589934
  Bounding box location: Location(x=303.606720, y=315.283234, z=6.110259)
  Bounding box extent: Vector3D(x=1.087687, y=0.564039, z=0.670259)
  Tags: Buildings
Building 1:
  ID: 11917469800758174349
  Bounding box location: Location(x=32.930000, y=211.193497, z=2.505811)
  Bounding box extent: Vector3D(x=1.087687, y=0.564039, z=0.670259)
  Tags: Buildings
Building 2:
  ID: 11282433653665041874
  Bounding box location: Location(x=157.300003, y=322.977478, z=2.709483)
  Bounding box extent: Vector3D(x=2.233471, y=0.972817, z=0.886711)
  Tags: Buildings
Building 3:
  ID: 1323511436800775095
  Bounding box location: Location(x=161.649994, y=322.977478, z=2.709483)
  Bounding box extent: Vector3D(x=2.233471, y=0.972817, z=0.886711)
  Tags: Buildings
Building 4:
  ID: 17417343498420407938
  Bounding box location: Location(x=238.144012, y=295.250000, z=2.353758)
  Bounding box extent: Vector3D(x=2.233471, y=0.972817, z=0.886711)
  Ta